<h1 style='text-align: center;'>Лабораторна робота 2</h1>

<p style='text-align: end;'><b>ФБ-21 Жиговець Олександр</b></p>

<br>

<p> 
<b>Мета роботи:</b> Ознайомитися з основними кроками по роботі з даними - workflow від постановки задачі до написання пояснювальної записки, зрозуміти постановку задачі та природу даних, над якими виконуються аналітичні операції.
</p>

<br>

<p>
<b>Основні поняття:</b> raw data, data preparation
</p>

<hr>


<h2 style="text-align:center;">Імпорт бібліотек</h2>

In [1]:
import datetime
import pandas as pd
import urllib.request

import glob

<hr>

<h2 style="text-align:center;">Завантаження файлів</h2>

In [9]:

def download_file(i, year1, year2):
    time_now = datetime.datetime.now()
    date_now = datetime.date.today()

    #Отримуємо всі csv файли в каталозі
    files = glob.glob('*.csv') 
        
    filename = f"{i}___{date_now.year}_{date_now.month}_{date_now.day}___{time_now.hour}_{time_now.minute}.csv"

    isExists = False
    for file in files:
        if file.startswith(f"{i}___"):
            isExists = True

    if isExists:
        print(f"VHI {i} exists in {filename}")        
    else:
        vhi_url = urllib.request.urlopen(f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={i}&year1={year1}&year2={year2}&type=Mean")

        with open(filename, 'wb') as out:
            out.write(vhi_url.read())

        print(f"VHI {i} is downloaded...")

year1 = 2000
year2 = 2020

for i in range (1,28):
    download_file(i, year1, year2)

VHI 1 is downloaded...
VHI 2 is downloaded...
VHI 3 is downloaded...
VHI 4 is downloaded...
VHI 5 is downloaded...
VHI 6 is downloaded...
VHI 7 is downloaded...
VHI 8 is downloaded...
VHI 9 is downloaded...
VHI 10 is downloaded...
VHI 11 is downloaded...
VHI 12 is downloaded...
VHI 13 is downloaded...
VHI 14 is downloaded...
VHI 15 is downloaded...
VHI 16 is downloaded...
VHI 17 is downloaded...
VHI 18 is downloaded...
VHI 19 is downloaded...
VHI 20 is downloaded...
VHI 21 is downloaded...
VHI 22 is downloaded...
VHI 23 is downloaded...
VHI 24 is downloaded...
VHI 25 is downloaded...
VHI 26 is downloaded...
VHI 27 is downloaded...


<hr>

<h2 style="text-align:center;">Зчитування даних у фрейм</h2>

In [10]:
def make_dataframe(path):
    frames = []
    headers = ["Year", "Week", "SMN", "SMT", "VCI", "TCI", "VHI", "empty"]

    #Отримуємо всі csv файли в каталозі
    files = glob.glob(path + '/*.csv') 

    for file in files:
        file = file.replace(f"{path}\\", "")
        province = int(file.split("_")[0])

        df = pd.read_csv(file, header=1, names = headers)
        df = df.drop(df.index[-1])
        
        #Конвертація років та тижнів в Integer, потрібно для процедур, де потрібно порівняти ці значення
        df.at[0, "Year"] = df.at[0, "Year"][9:] #Виправлення першого рядка
        df["Year"] = df["Year"].astype(int)
        df["Week"] = df["Week"].astype(int)

        df.insert(0, "pID", province, True)
        df = df.drop("empty", axis=1)
                
        frames.append(df)

    df = pd.concat(frames).drop_duplicates().reset_index(drop=True)

    #12 - Київ, 20 - Севастополь
    df = df.loc[(df.pID != 12) & (df.pID != 20)]

    return df



df = make_dataframe("D:\KPI\DataAnalys\Lab2")
print(df)

       pID  Year  Week    SMN     SMT    VCI    TCI    VHI
0       10  2000     1  0.033  260.29  21.28  40.64  30.96
1       10  2000     2  0.033  260.40  22.74  42.06  32.40
2       10  2000     3  0.036  261.40  28.74  39.77  34.26
3       10  2000     4  0.043  262.45  36.16  37.55  36.86
4       10  2000     5  0.049  264.25  39.26  35.74  37.50
...    ...   ...   ...    ...     ...    ...    ...    ...
29479    9  2020    48  0.159  277.21  66.52  16.25  41.37
29480    9  2020    49  0.150  275.06  65.68  19.44  42.55
29481    9  2020    50  0.144  274.00  66.17  18.69  42.43
29482    9  2020    51  0.138  272.73  66.79  19.59  43.19
29483    9  2020    52  0.132  271.44  67.24  21.90  44.57

[27300 rows x 8 columns]


<hr>

<h2 style="text-align:center;">Заміна id областей на вказані</h2>

In [11]:
def set_new_pid(dataframe):
    ids = {
        1: 22, # Черкаська
        2: 24, # Чернігівська
        3: 23, # Чернівецька
        4: 25, # Крим
        5: 3, # Дніпропетровська
        6: 4, # Донецька
        7: 8, # Івано-Франківська
        8: 19, # Харківська 
        9: 20, # Херсонська
        10: 21, # Хмельницька
        11: 9, # Київська
        13: 10, # Кіровоградська
        14: 11, # Луганська
        15: 12, # Львівська
        16: 13, # Миколаївська
        17: 14, # Одеська
        18: 15, # Полтавська
        19: 16, # Рівненська
        21: 17, # Сумська
        22: 18, # Терноспільська
        23: 6, # Закарпатська
        24: 1, # Віницька 
        25: 2, # Волинська
        26: 7, # Запорізька
        27: 5 # Житомирська 
    }

    df = dataframe.copy()
    df["pID"] = df["pID"].replace(ids)
    return df 

df = set_new_pid(df)



<hr>

<h2 style="text-align:center;">Реалізувати процедури для формування вибірок</h2>

<hr>

<p>
Ряд для області за вказаний рік, пошук екстремумів (min, max)
</p>

In [12]:
def min_max(df, year1 = None, year2 = None):
    result = []

    #Якщо задано діапазон, застосувати його
    if year1 != None:
        df = df[ df["Year"] >= year1 ]

    if year2 != None:
        df = df[ df["Year"] <= year2 ]

    grouped = df.groupby(["pID", "Year"])

    for (pid, year), group in grouped:
        min = group["VHI"].min()
        max = group["VHI"].max()

        result.append({"pID": pid, "Year": year, "Min": min, "Max": max})

    return pd.DataFrame(result)

print(min_max(df, 2000, 2010))

     pID  Year    Min    Max
0      1  2000  11.25  63.27
1      1  2001  36.50  72.03
2      1  2002  44.51  62.03
3      1  2003  -1.00  69.49
4      1  2004  -1.00  77.27
..   ...   ...    ...    ...
270   25  2006  32.53  64.97
271   25  2007  13.28  58.35
272   25  2008  24.03  69.15
273   25  2009  20.62  51.34
274   25  2010  32.24  57.08

[275 rows x 4 columns]


<hr>

<p> 
Ряд за вказаний діапазон років, виявити роки, протягом яких помірні посухи торкнулись більше вказаного відсотка областей по Україні 
</p>

In [13]:
#Посушливі роки (помірні)
def dry_years_m(df, perc, year1 = None, year2 = None):
    result = {}

    #Якщо задано діапазон, застосувати його
    if year1 != None:
        df = df[ df["Year"] >= year1 ]

    if year2 != None:
        df = df[ df["Year"] <= year2 ]

    grouped = df.groupby("Year")
    
    for year, group in grouped:
        provinces = group["pID"].nunique()
        dry_provinces = group[ (group["VHI"] > 15) & (group["VHI"] < 30) ]["pID"].nunique()

        if dry_provinces / provinces > perc / 100:
            result[year] = (dry_provinces, round((dry_provinces / provinces) * 100, 2))

    #Вивід
    print("  Помірні посухи  ")
    print("------------------")
    print(f"Years   %     prov")
    for year, (dry_provinces, percent) in result.items():
        print(f"{year}   {percent}    {dry_provinces}")

dry_years_m(df, 50, 2000, 2020)

  Помірні посухи  
------------------
Years   %     prov
2000   100.0    25
2015   60.0    15
2019   76.0    19
2020   60.0    15


<hr>

<p> 
Ряд за вказаний діапазон років, виявити роки, протягом яких екстремальні посухи торкнулись більше вказаного відсотка областей по Україні 
</p>

In [14]:
#Посушливі роки (екстремальні)
def dry_years_e(df, perc, year1, year2):
    result = {}
    #Якщо задано діапазон, застосувати його
    if year1 != None:
        df = df[ df["Year"] >= year1 ]

    if year2 != None:
        df = df[ df["Year"] <= year2 ]

    grouped = df.groupby("Year")

    for year, group in grouped:
        provinces = group["pID"].nunique()
        dry_provinces = group[group["VHI"] < 15 ]["pID"].nunique()

        if dry_provinces / provinces > perc / 100:
            result[year] = (dry_provinces, round((dry_provinces / provinces) * 100, 2))

    #Вивід
    print("Екстремальні  посухи  ")
    print(" ------------------")
    print(f" Years   %     prov")
    for year, (dry_provinces, percent) in result.items():
        print(f" {year}   {percent}    {dry_provinces}")

dry_years_e(df, 50, 2000, 2020)

Екстремальні  посухи  
 ------------------
 Years   %     prov
 2003   100.0    25
 2004   100.0    25
 2005   100.0    25


<hr>